run convolutional neural network for multi-classification or multi-output regression, input=images



In [ ]:
# import various libraries
import numpy as np
import csv
import pandas as pd
import argparse
import sklearn
import keras
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RepeatedKFold
from sklearn.preprocessing import OneHotEncoder
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten, MaxPooling2D, Dropout, BatchNormalization
from keras import backend
from pathlib import Path
import tensorflow as tf

In [ ]:
# parse arguments
dir = '/content/drive/MyDrive/FateTrack/0602/HCR_507_D2_3_w2/collections_2021616/' #directory where training/test data is saved
suff = 'Clu_epithelial' #data suffix (usually date)
out = '0621_Clu_C_kernel' #output suffix (usually same as suff)
n_epochs = 60 #number of epochs for training

In [ ]:
# get the dataset
def get_dataset(): #change this to load in data
    #X_train = pd.read_csv(dir + "train_" + suff + ".csv", sep = ',', header = None)
    #y_train = pd.read_csv(dir + "train_id_" + suff + ".csv", sep = ',', header = None)
    #X_test = pd.read_csv(dir + "test_" + suff + ".csv", sep = ',', header = None)
    #y_test = pd.read_csv(dir + "test_id_" + suff + ".csv", sep = ',', header = None)

    # X_train = np.load(dir + "nuclei" + suff + ".npy")
    # X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], X_train.shape[2], 1)
    # y_train = np.asarray(pd.read_csv(dir + "nuclei_id" + suff + ".csv", header = None))
    # encoder = OneHotEncoder(sparse=False)
    # y_train = encoder.fit_transform(y_train)
    # X_test = pd.DataFrame()
    # y_test = pd.DataFrame()
    # #y_test = encoder.transform(y_test)

    # train_test_split of full dataset
    X = np.load(dir + "nuclei_" + suff + ".npy")
    X = X.reshape(X.shape[0], X.shape[1], X.shape[2], 1)
    #y = np.asarray(pd.read_csv(dir + "nuclei_id_" + suff + ".csv", header = None))
    y = np.asarray(pd.read_csv(dir + "nuclei_id_" + suff + ".csv", header = None))
    encoder = OneHotEncoder(sparse=False)
    #y = encoder.fit_transform(y)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 2, stratify = y) # use stratify for classification
    return X_train, X_test, y_train, y_test, encoder

# get the model
def get_model(n_inputs, n_outputs):
    model = Sequential()
    model.add(Conv2D(20, kernel_size=5, activation='relu', input_shape=(60,60,1), padding = 'same')) #change input shape based on images
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size = (2,2)))
    #model.add(Dropout(0.25))
    model.add(Conv2D(60, kernel_size=4, activation='relu', padding = 'same'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size = (2,2)))
    #model.add(Dropout(0.25))
    model.add(Conv2D(100, kernel_size=3, activation='relu', padding = 'same'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size = (2,2)))
    #model.add(Dropout(0.25))
    model.add(Flatten()) #can also add another dense layer before output if you want
    model.add(Dense(500, activation = 'relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.5))
    model.add(Dense(50, activation = 'relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.5))
    #model.add(Dense(n_outputs, activation = 'sigmoid')) # binary classification
    #model.add(Dense(n_outputs, activation = 'softmax')) # multi-classification
    model.add(Dense(n_outputs, activation = 'relu')) # regression
    #model.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['categorical_accuracy']) # binary-classification
    #model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['categorical_accuracy']) # multi-classification
    model.compile(loss = 'mse', optimizer = 'adam', metrics = ['mse']) # regression
    return model

In [ ]:
# load dataset
X_train, X_test, y_train, y_test, encoder = get_dataset()
n_inputs, n_outputs = X_train.shape[0], y_train.shape[1]
X_train = tf.convert_to_tensor(X_train)
X_test = tf.convert_to_tensor(X_test)
y_train = tf.convert_to_tensor(y_train)
y_test = tf.convert_to_tensor(y_test)

# get model
model = get_model(n_inputs, n_outputs)
model.summary()
# fit the model on all data
history = model.fit(X_train, y_train, validation_data = (X_test, y_test), verbose = 2, epochs = n_epochs)#, batch_size = 25)
# intermediate_layer_model = keras.Model(inputs=model.input,
#                                        outputs=model.layers[-2].output)
# intermediate_output = intermediate_layer_model(keras.Input([X_train, y_train]))
# print(intermediate_output)

# # make a prediction for new data and write out to csv
y_pred = model.predict(X_test)
y_test_labels = encoder.inverse_transform(y_test)
y_pred_labels = encoder.inverse_transform(y_pred)
results = pd.DataFrame(np.hstack((y_pred_labels, y_test_labels)))
results.to_csv(dir + "results_" + out + ".csv")

# # # write accuracy and validation accuracy out to csv
# accuracy = pd.DataFrame(data=history.history['accuracy'])
# val_accuracy = pd.DataFrame(history.history['val_accuracy'])
# accuracy = pd.concat([accuracy, val_accuracy], axis=1)
# accuracy.to_csv(dir + 'accuracy_' + out + '.csv')

Model: "sequential_46"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_132 (Conv2D)          (None, 60, 60, 20)        520       
_________________________________________________________________
batch_normalization_70 (Batc (None, 60, 60, 20)        80        
_________________________________________________________________
max_pooling2d_127 (MaxPoolin (None, 30, 30, 20)        0         
_________________________________________________________________
conv2d_133 (Conv2D)          (None, 30, 30, 60)        19260     
_________________________________________________________________
batch_normalization_71 (Batc (None, 30, 30, 60)        240       
_________________________________________________________________
max_pooling2d_128 (MaxPoolin (None, 15, 15, 60)        0         
_________________________________________________________________
conv2d_134 (Conv2D)          (None, 15, 15, 100)     

TypeError: ignored